In [1]:
from pymongo import MongoClient, InsertOne, UpdateOne, DeleteOne

# Establish client connection
client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
courses_collection = db['courses']

# Bulk insert of courses with student enrollments
operations = [
    InsertOne({'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}),
    InsertOne({'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}),
    InsertOne({'course': 'History 201', 'enrollments': 20, 'department': 'History'}),
    InsertOne({'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics'})
]
courses_collection.bulk_write(operations)
print('Courses inserted succesfully')

Courses inserted succesfully


In [2]:
# Query for courses with enrollments greater than 20
for course in courses_collection.find({'enrollments': {'$gt': 20}}):
    print(course)

{'_id': ObjectId('690c100c719ff628841dff5a'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}
{'_id': ObjectId('690c100c719ff628841dff5b'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}


In [5]:
# Query courses in Computer Science or Mathematic Departments
for course in courses_collection.find({'department': {'$in': ['Computer Science', 'Mathematics']}}):
    print(course)

{'_id': ObjectId('690c100c719ff628841dff5a'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}
{'_id': ObjectId('690c100c719ff628841dff5b'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}


In [8]:
# Average enrollments per department using aggregation
pipeline = [
    {'$group': {'_id': '$department', 'average_enrollment': {'$avg': '$enrollments'}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': 'Computer Science', 'average_enrollment': 25.0}
{'_id': 'Physics', 'average_enrollment': 15.0}
{'_id': 'Mathematics', 'average_enrollment': 30.0}
{'_id': 'History', 'average_enrollment': 20.0}


In [9]:
# Maximum enrollment per department
pipeline = [
    {'$group': {'_id': '$department', 'max_enrollment': {'$max': '$enrollments'}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': 'Computer Science', 'max_enrollment': 25}
{'_id': 'Physics', 'max_enrollment': 15}
{'_id': 'Mathematics', 'max_enrollment': 30}
{'_id': 'History', 'max_enrollment': 20}


In [11]:
# Projection to rename fields
pipeline = [
    {'$project': {'course_name': '$course', 'department_name': '$department', 'enrollments':1}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': ObjectId('690c100c719ff628841dff5a'), 'enrollments': 30, 'course_name': 'Math 101', 'department_name': 'Mathematics'}
{'_id': ObjectId('690c100c719ff628841dff5b'), 'enrollments': 25, 'course_name': 'CS 102', 'department_name': 'Computer Science'}
{'_id': ObjectId('690c100c719ff628841dff5c'), 'enrollments': 20, 'course_name': 'History 201', 'department_name': 'History'}
{'_id': ObjectId('690c100c719ff628841dff5d'), 'enrollments': 15, 'course_name': 'Physics 202', 'department_name': 'Physics'}


In [15]:
# Adding enrollment category field based on enrollments
pipeline = [
    {'$addFields': {'enrollment_category': {'$cond': {'if': {'$gt': ['$enrollments', 20]}, 'then': 'high', 'else': 'low'}}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': ObjectId('690c100c719ff628841dff5a'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics', 'enrollment_category': 'high'}
{'_id': ObjectId('690c100c719ff628841dff5b'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science', 'enrollment_category': 'high'}
{'_id': ObjectId('690c100c719ff628841dff5c'), 'course': 'History 201', 'enrollments': 20, 'department': 'History', 'enrollment_category': 'low'}
{'_id': ObjectId('690c100c719ff628841dff5d'), 'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics', 'enrollment_category': 'low'}


In [16]:
# Bulk Update
update_operations = [
    UpdateOne({'enrollments': {'$gt': 20}}, {'$set': {'enrollment_category': 'high'}}),
    UpdateOne({'enrollments': {'$lte': 20}}, {'$set': {'enrollment_category': 'low'}})
]
courses_collection.bulk_write(update_operations)

print('Courses updated succesfully')

Matched count: 1
Modified count: 1


In [17]:
# Bulk Delete
delete_operations = [
    DeleteOne({'enrollments': {'$lt': 20}})
]
delete_result = courses_collection.bulk_write(delete_operations)

print(f"Deleted count: {delete_result.deleted_count}")

Deleted count: 0
